# Observación y tratamiento de las imágenes
Lo primeso que hemos hecho es colocar las imágenes por tipología en carpetas, que vamos a recortar y tratar de forma diferente:
### Cuadros-coleccion
Son cuadros que tienen marcos bien definidos, de imágenes religiosas.
Marcos: Dejo los marcos para mantener el estilo original.
Proporciones: Mantenemos las proporciones originales y redimensionamos al mismo ancho, ajustando la altura proporcionalmente.
### Cuadros-figuras
Son cuadros del barroco andino, sin marcos, y en algunos casos giradas y con formas no homogeneas
Marcos: No tienen marcos.
Proporciones: Mantenemos las proporciones originales y redimensionamos al mismo ancho, ajustando la altura proporcionalmente.
### Esculturas y esculturas-coleccion
Son esculturas religiosas y esculturas del barroco andino
Proporciones: Mantenemos las proporciones originales y redimensionamos al mismo ancho, ajustando la altura proporcionalmente.
### Murales 
Son pinturas de murales del barroco andino
Proporciones: Pueden ser recortados y cambiadas sus dimensiones. 

In [10]:
from PIL import Image, UnidentifiedImageError
import os

def is_image_file(filename):
    valid_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff")
    return filename.lower().endswith(valid_extensions)

def resize_image(image_path, output_path, width=None, height=None):
    try:
        with Image.open(image_path) as img:
            original_width, original_height = img.size
            if width and height:
                img = img.resize((width, height), Image.LANCZOS)
            elif width:
                ratio = width / float(original_width)
                new_height = int(ratio * original_height)
                img = img.resize((width, new_height), Image.LANCZOS)
            elif height:
                ratio = height / float(original_height)
                new_width = int(ratio * original_width)
                img = img.resize((new_width, height), Image.LANCZOS)
            img.save(output_path)
    except UnidentifiedImageError:
        print(f"Error: {image_path} is not a valid image file.")

# Función para procesar todas las imágenes en una carpeta
def process_folder(input_folder, output_folder, width=None, height=None):
    os.makedirs(output_folder, exist_ok=True)
    for image_name in os.listdir(input_folder):
        input_path = os.path.join(input_folder, image_name)
        output_path = os.path.join(output_folder, image_name)
        if is_image_file(image_name):
            resize_image(input_path, output_path, width, height)
        else:
            print(f"Skipping non-image file: {image_name}")

# Rutas de las carpetas de entrada y salida
input_base_folder = "./../data/raw"
output_base_folder = "./../data/processed"

# Carpetas y sus configuraciones específicas
folders_config = {
    "cuadros-coleccion": {"width": 512, "height": None},
    "cuadros-figuras": {"width": 512, "height": None},
    "esculturas": {"width": 512, "height": None},
    "esculturas-coleccion": {"width": 512, "height": None},
    "murales": {"width": 512, "height": 512}  # Ejemplo, ajusta según necesites
}

# Procesar cada carpeta con su configuración
for folder, config in folders_config.items():
    input_folder = os.path.join(input_base_folder, folder)
    output_folder = os.path.join(output_base_folder, folder)
    process_folder(input_folder, output_folder, width=config["width"], height=config["height"])

Skipping non-image file: .DS_Store


/Users/jesusmartinmejias/Library/Python/3.9/lib/python/site-packages/PIL/Image.py:3218: DecompressionBombWarning: Image size (101756928 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Skipping non-image file: .DS_Store
Skipping non-image file: .DS_Store


# Aumento de datos
Vamos a aumentar la muestra con algunas técnicas como cambio de color, espejado, rotación etc en cada caso. Como cada caso es distinto aplicaremos diferentes técnicas

In [2]:
import Augmentor
import os

def augment_data(input_folder, output_folder, operations, sample_size=1000):
    p = Augmentor.Pipeline(input_folder, output_directory=output_folder)
    
    # Añadir operaciones de aumento de datos según las especificaciones
    for operation in operations:
        if operation == "rotate":
            p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
        elif operation == "flip_lr":
            p.flip_left_right(probability=0.5)
        elif operation == "flip_tb":
            p.flip_top_bottom(probability=0.5)
        elif operation == "random_contrast":
            p.random_contrast(probability=0.5, min_factor=0.7, max_factor=1.3)
        elif operation == "random_brightness":
            p.random_brightness(probability=0.5, min_factor=0.7, max_factor=1.3)
        elif operation == "random_color":
            p.random_color(probability=0.5, min_factor=0.7, max_factor=1.3)
        elif operation == "crop_random":
            p.crop_random(probability=0.5, percentage_area=0.8)
    
    p.sample(sample_size)

# Rutas de las carpetas de entrada y salida
input_base_folder = "./../data/processed"
output_base_folder = "./../data/augmented"

# Configuraciones específicas para cada carpeta
folders_config = {
    "cuadros-coleccion": ["random_contrast", "random_brightness", "random_color"],
    "cuadros-figuras": ["rotate", "flip_lr", "flip_tb", "random_contrast", "random_brightness", "random_color", "crop_random"],
    "esculturas": ["flip_lr", "random_contrast", "random_brightness", "random_color"],
    "esculturas-coleccion": ["flip_lr", "random_contrast", "random_brightness", "random_color"],
    "murales": ["rotate", "flip_lr", "flip_tb", "random_contrast", "random_brightness", "random_color", "crop_random"]
}

# Procesar cada carpeta con sus operaciones específicas
for folder, operations in folders_config.items():
    input_folder = os.path.join(input_base_folder, folder)
    output_folder = os.path.join(output_base_folder, folder)
    augment_data(input_folder, output_folder, operations, sample_size=1000)


Initialised with 22 image(s) found.
Output directory set to ./../data/processed/cuadros-coleccion/./../data/augmented/cuadros-coleccion.

Processing <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x679 at 0x111585610>: 100%|██████████| 1000/1000 [00:10<00:00, 92.44 Samples/s]


Initialised with 36 image(s) found.
Output directory set to ./../data/processed/cuadros-figuras/./../data/augmented/cuadros-figuras.

Processing <PIL.Image.Image image mode=RGB size=512x287 at 0x10E8AF200>: 100%|██████████| 1000/1000 [00:10<00:00, 98.74 Samples/s]                  


Initialised with 3 image(s) found.
Output directory set to ./../data/processed/esculturas/./../data/augmented/esculturas.

Processing <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=512x384 at 0x10E8AFB60>: 100%|██████████| 1000/1000 [00:08<00:00, 123.23 Samples/s]


Initialised with 7 image(s) found.
Output directory set to ./../data/processed/esculturas-coleccion/./../data/augmented/esculturas-coleccion.

Processing <PIL.Image.Image image mode=RGB size=512x716 at 0x10E95AD20>: 100%|██████████| 1000/1000 [00:09<00:00, 105.72 Samples/s]                 


Initialised with 56 image(s) found.
Output directory set to ./../data/processed/murales/./../data/augmented/murales.

Processing <PIL.Image.Image image mode=RGB size=512x512 at 0x10E948650>: 100%|██████████| 1000/1000 [00:09<00:00, 110.43 Samples/s]                 
